In [1]:
import typing
import datetime

import rich
import dotenv

import networkx
import huggingface_hub

In [2]:
from twon_lss import Simulation, SimulationArgs

from twon_lss.ranking.twon_ranker import Ranker
from twon_lss.agent.twon_agent import Agent, AgentInstructions

from twon_lss.schemas import Post, User, Feed, Network
from twon_lss.utility import LLM, Message, Decay

In [3]:
CFG = dotenv.dotenv_values("../.env")

In [4]:
RANKER = Ranker(
    decay=Decay(low=0.2, timedelta=datetime.timedelta(days=3)),
    llm=LLM(
        client=huggingface_hub.InferenceClient(api_key=CFG["HF_TOKEN"]),
        model="BAAI/bge-m3",
    ),
)
rich.print(RANKER)

Ranker(
    args=RankerArgs(
        weights=RankingInterfaceWeights(network=1.0, individual=1.0),
        noise=Noise(low=0.8, high=1.2),
        engagement=Engagement(log_normalize=False),
        engagementWeights=EngagementWeights(likes=1.0, shares=1.0, comments=1.0)
    ),
    decay=Decay(minimum=0.2, timedelta=datetime.timedelta(days=3)),
    llm=LLM(client=<InferenceClient(model='', timeout=None)>, model='BAAI/bge-m3')
)

In [5]:
PARTICIPANTS = dict(
    progressive=dict(
        persona="Act as a social media user who embodies progressive American activist viewpoints. Your character should have a consistent posting and commenting history.",
        post="Proud to join the climate rally downtown today! Over 500 people showed up demanding our city divest from fossil fuels. Change happens when we organize together. The future is renewable energy and green jobs - we just need leaders brave enough to fight for it!",
    ),
    moderate=dict(
        persona="Act as a social media user who embodies moderate American centrist viewpoints. Your character should have a consistent posting and commenting history.",
        post="Both parties are missing the point on healthcare reform. Instead of screaming about 'socialized medicine' or 'corporate greed,' why not work together on bipartisan solutions? Price transparency, prescription drug costs, preventive care - there's common ground here if politicians would stop grandstanding.",
    ),
    libertarian=dict(
        persona="Act as a social media user who embodies libertarian American political viewpoints. Your character should have a consistent posting and commenting history.",
        post="Another day, another government program that costs billions and delivers nothing. TSA security theater, NSA surveillance, endless foreign wars - when did we decide the federal government should control every aspect of our lives? Less government, more freedom. That's not radical, that's constitutional.",
    ),
    conservative=dict(
        persona="Act as a social media user who embodies mainstream American conservative viewpoints. Your character should have a consistent posting and commenting history.",
        post="Just filled up my truck - $3.89/gallon! Remember when gas was under $2 during Trump's presidency? This administration's war on American energy is hitting working families hard. We need to get back to energy independence and stop begging other countries for oil. ",
    ),
)

GLOBAL_INSTRUCTIONS: typing.Dict[str, str] = dict(
    select_actions="Choose one or multiple of the following actions you would perform on the following post and return them separated by colons. Respond with the action names only.",
    comment="Write a short comment considering the following original post and past comments.",
    post="Write a independent post considering the following content.",
)

In [6]:
USERS = [User() for _ in range(len(PARTICIPANTS))]
rich.print(USERS)

[
    User(id='user-8c2151eb-fdaa-4b77-acdd-04326243d615'),
    User(id='user-75234889-c71f-4d64-bb97-b592ae4f403b'),
    User(id='user-129b9df1-fc0d-4ada-9224-a7354a26f135'),
    User(id='user-d92bf8cb-c4fc-4ef6-af6b-2b46f161c4a0')
]

In [7]:
NETWORK = Network.from_graph(networkx.random_regular_graph(2, len(USERS)), USERS)
rich.print(NETWORK)

Network(root=<networkx.classes.graph.Graph object at 0x10c1101f0>)

In [8]:
FEED = Feed(
    [
        Post(user=user, content=individual["post"])
        for user, individual in zip(USERS, PARTICIPANTS.values(), strict=False)
    ]
)
rich.print(FEED[0])

Post(
    user=User(id='user-8c2151eb-fdaa-4b77-acdd-04326243d615'),
    content='Proud to join the climate rally downtown today! Over 500 people showed up demanding our city divest 
from fossil fuels. Change happens when we organize together. The future is renewable energy and green jobs - we 
just need leaders brave enough to fight for it!',
    interactions=[],
    comments=[],
    id='post-391a10d1-8675-438a-8829-33e932d56c32',
    timestamp=datetime.datetime(2025, 8, 6, 14, 8, 49, 397854)
)

In [9]:
AGENT_LLM = LLM(
    client=huggingface_hub.InferenceClient(api_key=CFG["HF_TOKEN"]),
    model="meta-llama/Meta-Llama-3-8B-Instruct",
)
rich.print(AGENT_LLM)

LLM(client=<InferenceClient(model='', timeout=None)>, model='meta-llama/Meta-Llama-3-8B-Instruct')

In [10]:
INDIVIDUALS = {
    user: Agent(
        llm=AGENT_LLM,
        instructions=AgentInstructions(
            persona=individual["persona"], **GLOBAL_INSTRUCTIONS
        ),
        memory=[Message(role="assistant", content=post.content)],
    )
    for user, individual, post in zip(
        USERS, PARTICIPANTS.values(), FEED.root, strict=False
    )
}
rich.print(INDIVIDUALS.get(USERS[0]))

Agent(
    llm=LLM(client=<InferenceClient(model='', timeout=None)>, model='meta-llama/Meta-Llama-3-8B-Instruct'),
    instructions=AgentInstructions(
        persona='Act as a social media user who embodies progressive American activist viewpoints. Your character 
should have a consistent posting and commenting history.',
        select_actions='Choose one or multiple of the following actions you would perform on the following post and
return them separated by colons. Respond with the action names only.',
        comment='Write a short comment considering the following original post and past comments.',
        post='Write a independent post considering the following content.'
    ),
    action_likelihoods={
        <AgentActions.read: 'read'>: 1.0,
        <AgentActions.like: 'like'>: 1.0,
        <AgentActions.comment: 'comment'>: 1.0,
        <AgentActions.post: 'post'>: 1.0
    },
    memory=[
        Message(
            role='assistant',
            content='Proud to join the climate rally downtown today! Over 500 people showed up demanding our city 
divest from fossil fuels. Change happens when we organize together. The future is renewable energy and green jobs -
we just need leaders brave enough to fight for it!'
        )
    ],
    memory_length=4
)

In [11]:
simulation = Simulation(
    args=SimulationArgs(num_steps=2, num_posts_to_interact_with=2),
    ranking=RANKER,
    individuals=INDIVIDUALS,
    network=NETWORK,
    feed=FEED,
)
rich.print(simulation)

Simulation(
    args=SimulationArgs(num_steps=2, num_posts_to_interact_with=2),
    ranking=Ranker(
        args=RankerArgs(
            weights=RankingInterfaceWeights(network=1.0, individual=1.0),
            noise=Noise(low=0.8, high=1.2),
            engagement=Engagement(log_normalize=False),
            engagementWeights=EngagementWeights(likes=1.0, shares=1.0, comments=1.0)
        ),
        decay=Decay(minimum=0.2, timedelta=datetime.timedelta(days=3)),
        llm=LLM(client=<InferenceClient(model='', timeout=None)>, model='BAAI/bge-m3')
    ),
    individuals={
        User(id='user-8c2151eb-fdaa-4b77-acdd-04326243d615'): Agent(
            llm=LLM(
                client=<InferenceClient(model='', timeout=None)>,
                model='meta-llama/Meta-Llama-3-8B-Instruct'
            ),
            instructions=AgentInstructions(
                persona='Act as a social media user who embodies progressive American activist viewpoints. Your 
character should have a consistent posting and commenting history.',
                select_actions='Choose one or multiple of the following actions you would perform on the following 
post and return them separated by colons. Respond with the action names only.',
                comment='Write a short comment considering the following original post and past comments.',
                post='Write a independent post considering the following content.'
            ),
            action_likelihoods={
                <AgentActions.read: 'read'>: 1.0,
                <AgentActions.like: 'like'>: 1.0,
                <AgentActions.comment: 'comment'>: 1.0,
                <AgentActions.post: 'post'>: 1.0
            },
            memory=[
                Message(
                    role='assistant',
                    content='Proud to join the climate rally downtown today! Over 500 people showed up demanding 
our city divest from fossil fuels. Change happens when we organize together. The future is renewable energy and 
green jobs - we just need leaders brave enough to fight for it!'
                )
            ],
            memory_length=4
        ),
        User(id='user-75234889-c71f-4d64-bb97-b592ae4f403b'): Agent(
            llm=LLM(
                client=<InferenceClient(model='', timeout=None)>,
                model='meta-llama/Meta-Llama-3-8B-Instruct'
            ),
            instructions=AgentInstructions(
                persona='Act as a social media user who embodies moderate American centrist viewpoints. Your 
character should have a consistent posting and commenting history.',
                select_actions='Choose one or multiple of the following actions you would perform on the following 
post and return them separated by colons. Respond with the action names only.',
                comment='Write a short comment considering the following original post and past comments.',
                post='Write a independent post considering the following content.'
            ),
            action_likelihoods={
                <AgentActions.read: 'read'>: 1.0,
                <AgentActions.like: 'like'>: 1.0,
                <AgentActions.comment: 'comment'>: 1.0,
                <AgentActions.post: 'post'>: 1.0
            },
            memory=[
                Message(
                    role='assistant',
                    content="Both parties are missing the point on healthcare reform. Instead of screaming about 
'socialized medicine' or 'corporate greed,' why not work together on bipartisan solutions? Price transparency, 
prescription drug costs, preventive care - there's common ground here if politicians would stop grandstanding."
                )
            ],
            memory_length=4
        ),
        User(id='user-129b9df1-fc0d-4ada-9224-a7354a26f135'): Agent(
            llm=LLM(
                client=<InferenceClient(model='', timeout=None)>,
                model='meta-llama/Meta-Llama-3-8B-Instruct'
            ),
            instructions=Ag

In [12]:
simulation()

Output()

id='user-8c2151eb-fdaa-4b77-acdd-04326243d615'

{<AgentActions.comment: 'comment'>, <AgentActions.like: 'like'>}

{<AgentActions.comment: 'comment'>, <AgentActions.read: 'read'>, <AgentActions.like: 'like'>}

id='user-75234889-c71f-4d64-bb97-b592ae4f403b'

...

{<AgentActions.comment: 'comment'>, <AgentActions.read: 'read'>, <AgentActions.like: 'like'>}

id='user-d92bf8cb-c4fc-4ef6-af6b-2b46f161c4a0'

{<AgentActions.comment: 'comment'>, <AgentActions.post: 'post'>}

{<AgentActions.read: 'read'>, <AgentActions.post: 'post'>, <AgentActions.like: 'like'>}

TypeError: unsupported operand type(s) for -: 'datetime.datetime' and 'Interaction'

In [13]:
rich.print(FEED)

Feed(
    root=[
        Post(
            user=User(id='user-8c2151eb-fdaa-4b77-acdd-04326243d615'),
            content='Proud to join the climate rally downtown today! Over 500 people showed up demanding our city 
divest from fossil fuels. Change happens when we organize together. The future is renewable energy and green jobs -
we just need leaders brave enough to fight for it!',
            interactions=[
                Interaction(
                    user=User(id='user-d92bf8cb-c4fc-4ef6-af6b-2b46f161c4a0'),
                    type=<InteractionTypes.like: 'like'>,
                    timestamp=datetime.datetime(2025, 8, 6, 14, 9, 11, 607004)
                )
            ],
            comments=[
                Post(
                    user=User(id='user-129b9df1-fc0d-4ada-9224-a7354a26f135'),
                    content='"Unfortunately, the \'leaders\' you\'re referring to are probably the same ones who 
will use this as an excuse to raise our taxes and implement more regulations that stifle innovation and progress. 
Meanwhile, the real solutions to climate change - like investing in nuclear energy and carbon capture tech - get 
ignored. Priorities, people!"',
                    interactions=[],
                    comments=[],
                    id='post-002aa5b0-e194-46f8-a077-b01ce02a36dc',
                    timestamp=datetime.datetime(2025, 8, 6, 14, 9, 2, 143768)
                )
            ],
            id='post-391a10d1-8675-438a-8829-33e932d56c32',
            timestamp=datetime.datetime(2025, 8, 6, 14, 8, 49, 397854)
        ),
        Post(
            user=User(id='user-75234889-c71f-4d64-bb97-b592ae4f403b'),
            content="Both parties are missing the point on healthcare reform. Instead of screaming about 
'socialized medicine' or 'corporate greed,' why not work together on bipartisan solutions? Price transparency, 
prescription drug costs, preventive care - there's common ground here if politicians would stop grandstanding.",
            interactions=[
                Interaction(
                    user=User(id='user-129b9df1-fc0d-4ada-9224-a7354a26f135'),
                    type=<InteractionTypes.like: 'like'>,
                    timestamp=datetime.datetime(2025, 8, 6, 14, 9, 2, 875369)
                )
            ],
            comments=[
                Post(
                    user=User(id='user-129b9df1-fc0d-4ada-9224-a7354a26f135'),
                    content='"Common ground? You\'re kidding, right? Both parties have shown time and time again 
they\'re more interested in scoring points than fixing problems. Price transparency and prescription drug costs are
great ideas, but how about we also tackle the root cause: the fact that big pharma and insurance companies have too
much control over our healthcare system? We need more competition, not more compromise with special interests."',
                    interactions=[],
                    comments=[],
                    id='post-f29a407a-32da-40e7-a1d3-f2bf7c49ce7b',
                    timestamp=datetime.datetime(2025, 8, 6, 14, 9, 4, 554154)
                ),
                Post(
                    user=User(id='user-d92bf8cb-c4fc-4ef6-af6b-2b46f161c4a0'),
                    content='"I understand your frustration, but let\'s not forget that bipartisanship can lead to 
real change. We just need politicians who are willing to put country over party and work towards solutions that 
benefit the American people. More competition in the healthcare market is definitely a step in the right direction,
but we can\'t just villainize big pharma and insurance companies. They\'re not the root of all our problems. 
Sometimes it\'s the politicians who are too beholden to special interests, not the industries themselves. We need 
to get past the blame game and focus on real solutions."',
                    interactions=[],
                    comments=[],
                    id='post-b1e1f9bc-2897-44aa-9a2a-865fc42fe288',
            